In [0]:
dbutils.widgets.text("File_name","BLANK","Enter File name")
File_name = dbutils.widgets.get("File_name")
print(f"File_name: {File_name}")

In [0]:
dbutils.widgets.text("Database_name","BLANK","Enter Database name")
Database_name = dbutils.widgets.get("Database_name")
print(f"Database_name: {Database_name}")

In [0]:
import pandas as pd 
import os

In [0]:
os.getcwd()

In [0]:
Table_Detail = pd.read_csv(File_name)
display(Table_Detail)

In [0]:
Table_Detail['Column_List'] = Table_Detail['Column_Name'] + ' ' + Table_Detail['Data_Type']
Table_Detail_new = Table_Detail[['Table_Name','Column_List']]
display(Table_Detail_new)

Table_Detail_new2 = Table_Detail_new\
.groupby('Table_Name')['Column_List']\
.apply(lambda x: '(' + ','.join(x.astype(str)) + ')')\
.reset_index()
display(Table_Detail_new2)


In [0]:
def createTable(Table_Name, Column_List):
    spark.sql(f"create database if not exists {Database_name}")
    spark.sql(f"drop table if exists {Database_name}.{Table_Name}")
    spark.sql(f"create table {Database_name}.{Table_Name} {Column_List}")

In [0]:
for index, row in Table_Detail_new2.iterrows():
    createTable(row['Table_Name'], row['Column_List'])

In [0]:
#PySpark implements distributed data processing (consumes more resources). Thus, use PySpark only for huge data. 

from pyspark.sql import functions as F

Table_Detail['Column_List'] = Table_Detail['Column_Name'] + ' ' + Table_Detail['Data_Type']
Table_Detail_new = spark.createDataFrame(Table_Detail[['Table_Name','Column_List']]) 
display(Table_Detail_new)

Table_Detail_new2 = Table_Detail_new.groupby('Table_Name')\
.agg(F.concat_ws(',',F.collect_list('Column_List'))\
     .alias('Column_List'))\
     .withColumn('Column_List', F.concat(F.lit("("), F.col("Column_List"), F.lit(")")))
display(Table_Detail_new2)